In [ ]:
import seaborn as sns

import sklearn
from sklearn.cluster.bicluster import SpectralBiclustering
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import consensus_score

import scipy.linalg
from functools import partial
%pylab
%matplotlib inline
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import scipy.sparse as sparse
from scipy import stats
from __future__ import print_function
import gc
from tqdm import tqdm_notebook as tqdm
from __future__ import division
import pandas as pd
import seaborn as sb
import os
from scipy.spatial import distance
from scipy.cluster import hierarchy
from pandas import DataFrame

import os, sys, datetime
LF_CODE_PATH = os.path.expanduser('~/projects/LFAnalyze/code')
FT_CODE_PATH = os.path.expanduser('~/projects/fishTrax/code/analysis/')
FD_CODE_PATH = os.path.expanduser('~/projects/fish_despair_notebooks/src/')
sys.path.insert(0,LF_CODE_PATH)
sys.path.insert(0,FT_CODE_PATH)
sys.path.insert(0,FD_CODE_PATH)

import passivity_2p_imaging_utils as p2putils
import visualization_utils as vizutil
reload(p2putils)
tmp_dir = '/tmp/'
all_data = p2putils.get_all_datasets(tmp_dir=tmp_dir)

f = all_data['e'][2]

import tensorflow as tf
# tf.enable_eager_execution()


# df, sig = f.get_cnmf_roi_table_and_signals()
M = f.get_signals_raw(z=None)

# data = np.load("../cnmf_f01555.npz")
# cnmf = data['cnmf'].astype(np.float32)
# raw = data['raw'].astype(np.float32)
# del data

def moving_average(a, n=3):
    # a is a signal
    ret = np.cumsum(a,0) # sum over time
    ret[n:] = ret[n:] - ret[:-n] # diff of n samples back
    rm = ret[n - 1:] / n
    pad_start = np.full((n-1,rm.shape[1]), rm[0])
    return np.vstack([pad_start, rm])

def ewma(data,span):
    "exponential weighted moving average."
    df = DataFrame(data)
    return df.ewm(span).mean().values

def df_f(x,ma_window=6,span=6):
    u = moving_average(x,ma_window)
    return ewma((x - u)/u, span)

In [ ]:
# neurons = sig.T
neurons = M.T

# neuron_ids = np.sort(np.argsort(neurons.std(0))[-5000:])
df = f.get_roi_table()
# df = df.iloc[neuron_ids]
# df.reset_index(drop=True, inplace=True)
# neurons = neurons[:,neuron_ids]

from scipy import signal

neurons = signal.decimate(neurons,4, axis=0)

neurons = df_f(neurons).astype(np.float32)
neurons = (neurons - neurons.mean(0))/(neurons.std(0)+1e-8)

In [ ]:
reg_set = ['l_LHb','r_LHb']

#get boolean ndx of all rois not in regions set.
df = f.get_roi_table()
bndx = np.ones(len(df),dtype=np.bool)
for reg in reg_set:
    bndx = bndx & ~df['in_'+reg]

#get the x,y,z location of all neurons (units um)
x = np.array(df.centroid_x) * f.um_per_pixel
y = np.array(df.centroid_y) * f.um_per_pixel
z = np.array(df.z) * f.z_spacing_um

#cube size (um)
cube_size = [50,50,30]

#take origin as point between the hb.
x0 = x[df['in_l_LHb'] | df['in_r_LHb']].mean()
y0 = y[df['in_l_LHb'] | df['in_r_LHb']].mean()
z0 = z[df['in_l_LHb'] | df['in_r_LHb']].mean()

#get edges of boxes to be considered
x_edges = np.hstack([np.arange(x0,-cube_size[0],-cube_size[0])[::-1],np.arange(x0,x.max()+cube_size[0],cube_size[0])[1:]])
y_edges = np.hstack([np.arange(y0,-cube_size[1],-cube_size[1])[::-1],np.arange(y0,y.max()+cube_size[1],cube_size[1])[1:]])
z_edges = np.hstack([np.arange(z0,-cube_size[2],-cube_size[2])[::-1],np.arange(z0,z.max()+cube_size[2],cube_size[2])[1:]])

#build list of roi_sets with a box of rois held out from each set.  the first element has no rois heldout
bndx_sets = []
bndx_sets.append(bndx)

import itertools
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return itertools.izip(a, b)

for xs,xe in pairwise(x_edges):
    for ys,ye in pairwise(y_edges):
        for zs,ze in pairwise(z_edges):
            bndx_sets.append(bndx & ~((x>xs) & (x<xe) & (y>ys) & (y<ye) & (z>zs) & (z<ze)))

In [ ]:
reg_set

In [ ]:
from itertools import product

In [ ]:
from joblib import Parallel, delayed

In [ ]:
neurons.shape

In [ ]:
nT, N = neurons.shape
Y_mask = np.ones([2,N], dtype=bool)
Y_mask[0] = df['in_'+reg_set[0]]
Y_mask[1] = df['in_'+reg_set[1]]

In [ ]:
lag = 1
y = neurons[lag:,Y_mask[0]]
x_null = np.concatenate([neurons[0:-lag], np.ones([neurons.shape[0]-lag,1], dtype=np.float32)],1)
nbeta, n_residuals, _, _ = np.linalg.lstsq(x_null, y)
n_residuals = ((y - np.matmul(x_null,nbeta))**2).sum()
n_residuals

In [ ]:
def bndx_granger(neurons, Y_mask, X_mask, lag=1):
    try:
        nT, N = neurons.shape
        y = neurons[lag:,Y_mask]
        x_granger = np.concatenate([neurons[0:-lag,X_mask], np.ones([nT-lag,1], dtype=np.float32)],1)
        gbeta, g_residuals, _, _ = np.linalg.lstsq(x_granger, y)
        g_residuals = ((y - np.matmul(x_null,nbeta))**2).sum()
        return g_residuals
    except:
        return "Error"
    
g_residuals = Parallel(n_jobs=32)(delayed(bndx_granger)(neurons, y_mask, x_mask) for (y_mask, x_mask) in product(Y_mask, bndx_sets))

In [ ]:
g_residuals

In [ ]:
def construct_bndx_batches(bndx_sets, reg_set, X, lag=1, bias=True):
    """constructs  len(bndx_sets) x len(reg_set) x nT-lag x nBeta. [i, j] index is granger model of influence from i -> j.
    """
    nT, N = X.shape
    bz = len(bndx_sets)
    N_regions = len(reg_set)
    granger = np.ones([bz, N_regions, nT-lag, N+bias])
    null = np.ones([bz, N_regions, nT-lag, N+bias])
    newY = np.ones([bz, N_regions, nT-lag, 1])
    Y = X[lag:]
    # time x neuron x lag
    l = 0
    X_lag = np.concatenate([X[l:-(lag-l)][:,:,None] for l in reversed(range(lag))], 2)
    biasTerm = np.ones([nT-lag,1])
    for i, n1 in enumerate(range(startIdx,min(N,endIdx))):
        for j, n2 in enumerate(range(N)):
            if bias:
#                 print([x.shape for x in [X_lag[:,n1], X_lag[:,n2], biasTerm]])
                grow = np.concatenate([X_lag[:,n1], X_lag[:,n2], biasTerm], 1)
                nrow = np.concatenate([X_lag[:,n2], biasTerm], 1)
            else:
                grow = np.concatenate([X_lag[:,n1], X_lag[:,n2]], 1)
                nrow = np.concatenate([X_lag[:,n2]], 1)
#             print(i,j)
            granger[i*N+j] = grow
            null[i*N+j] = nrow
            newY[i*N+j] = Y[:,[n2]]
    return newY, granger, null

In [ ]:
def construct_pairwise_batches(X, lag=1, startIdx=None, endIdx=None, bias=True):
    """constructs batch x nT-lag x nBeta. i*ncol+j index is granger model of influence from i->j.
    
    Use startIdx and endIdx to choose a certain number of rows,
    eg startIdx=0, endIdx=5, X.shape[1]=100 would be a batch of 100^2=10,000.
    
    WARNING: if endIdx>N, will fill with 1s so batch stays same size."""
    
    nT, N = X.shape
    if endIdx:
        bz = (endIdx - startIdx) * N
    else:
        startIdx=0
        endIdx=N
        bz = N**2
#     print([bz, nT-lag, 2*lag+bias])
    granger = np.ones([bz, nT-lag, 2*lag+bias])
    null = np.ones([bz, nT-lag, lag+bias])
    newY = np.ones([bz, nT-lag, 1])
    Y = X[lag:]
    # time x neuron x lag
    l = 0
    X_lag = np.concatenate([X[l:-(lag-l)][:,:,None] for l in reversed(range(lag))], 2)
    biasTerm = np.ones([nT-lag,1])
    for i, n1 in enumerate(range(startIdx,min(N,endIdx))):
        for j, n2 in enumerate(range(N)):
            if bias:
#                 print([x.shape for x in [X_lag[:,n1], X_lag[:,n2], biasTerm]])
                grow = np.concatenate([X_lag[:,n1], X_lag[:,n2], biasTerm], 1)
                nrow = np.concatenate([X_lag[:,n2], biasTerm], 1)
            else:
                grow = np.concatenate([X_lag[:,n1], X_lag[:,n2]], 1)
                nrow = np.concatenate([X_lag[:,n2]], 1)
#             print(i,j)
            granger[i*N+j] = grow
            null[i*N+j] = nrow
            newY[i*N+j] = Y[:,[n2]]
    return newY, granger, null

In [ ]:
def f_value(resA, resB, n_params_A, n_params_B, n):
    return ( (resA-resB)/(n_params_B-n_params_A) ) / (resB/(n-n_params_B))

class Granger():
    def __init__(self, x_granger_shape, x_null_shape, y_shape, sess, l2=0.0):
        self.x_granger = tf.placeholder(dtype=tf.float32, shape=x_granger_shape)
        self.x_null = tf.placeholder(dtype=tf.float32, shape=x_null_shape)
        self.y = tf.placeholder(dtype=tf.float32, shape=y_shape)
        self.n_params_granger = x_granger_shape[-1]
        self.n_params_null = x_null_shape[-1]
        self.l2 = l2
        self.sess = sess
        
    def granger(self, X, Y):
        gbeta = tf.matrix_solve_ls(self.x_granger, self.y, l2_regularizer=self.l2, fast=False)
        g_residuals = tf.reduce_sum(((tf.matmul(self.x_granger,gbeta) - self.y)**2)[:,:,0], 1)
        gbeta, g_residuals = self.sess.run([gbeta, g_residuals],
                                      feed_dict={self.x_granger: X, self.y: Y})
        return gbeta, g_residuals
    
    def null(self, X, Y):
        gbeta = tf.matrix_solve_ls(self.x_null, self.y, l2_regularizer=self.l2, fast=False)
        g_residuals = tf.reduce_sum(((tf.matmul(self.x_null,gbeta) - self.y)**2)[:,:,0], 1)
        gbeta, g_residuals = self.sess.run([gbeta, g_residuals],
                                      feed_dict={self.x_null: X, self.y: Y})
        return gbeta, g_residuals

def pairwise_granger_f_val(neurons, lag=1, bz=10):
    nT, N = neurons.shape
    Y, x_granger, x_null = construct_pairwise_batches(neurons,lag,0,bz)
    fvals = np.zeros([N,N])
    with tf.Session() as sess:
        granger = Granger(x_granger.shape, x_null.shape, Y.shape, sess=sess, l2=0.0)
        for start in tqdm(range(0,N,bz)):
            end = start+bz
            Y, x_granger, x_null = construct_pairwise_batches(neurons,1,start,end)
            gbeta, g_residuals = granger.granger(x_granger, Y)
            nbeta, n_residuals = granger.null(x_null, Y)
            batch_fvals = f_value(g_residuals, n_residuals, granger.n_params_granger, granger.n_params_null, N)
            true_end = min(start+bz,N)
            batch_fvals = batch_fvals.reshape(-1,N)
            if start+bz>N:
                batch_true_end = N-start
                batch_fvals = batch_fvals[:batch_true_end]
            fvals[start:true_end] = batch_fvals
    return fvals

In [ ]:
f_vals = pairwise_granger_f_val(neurons, 1, 10)

plt.imshow(f_vals)
plt.colorbar()

In [ ]:
# f_vals = np.load("/data2/trained_models/181002_fvals.npy")
np.save("/data2/trained_models/181002_fvals_4x_decimate.npy",f_vals)

In [ ]:
def cm_cycle(i):
    i = i - 1
    cmap = plt.get_cmap("tab10")
    color = cmap(i%10)
    if i%10==7:
        color = (color[0],color[1],color[2]+0.4)
    return color

In [ ]:
row_corr = np.corrcoef(f_vals)

row_linkage = hierarchy.linkage(
    distance.pdist(row_corr), method='ward')

In [ ]:
f_vals.T.shape

In [ ]:
col_corr = np.corrcoef(f_vals.T)
col_linkage = hierarchy.linkage(
    distance.pdist(col_corr.T), method='ward')

In [ ]:
g_nclust = 10
row_clusters = hierarchy.fcluster(row_linkage,g_nclust,criterion='maxclust')
col_clusters = hierarchy.fcluster(col_linkage,g_nclust,criterion='maxclust')

In [ ]:
vmax = max(np.abs(np.percentile(f_vals,0.005)), np.abs(np.percentile(f_vals,0.995)))

In [ ]:
vmax

In [ ]:
cm = sb.clustermap(f_vals, row_linkage=row_linkage, row_colors=[cm_cycle(c) for c in row_clusters],
        col_linkage=col_linkage, col_colors=[cm_cycle(c) for c in col_clusters],
        figsize=(20, 20),cmap="RdBu_r", vmin=-vmax,vmax=vmax)

In [ ]:
plt.hist(f_vals.reshape(-1), np.linspace(-10,30,50));

In [ ]:
Z = [0,2,4,6,8,10]
# Z = [4]
nZ = len(Z)
back_img = []
for z in Z:
    back_img.append(np.power(f.get_tif_rasl_as_vol(z,range(1,200)).mean(axis=2),.4))
    

In [ ]:
# nrows = len(row_clust_to_plot)
# nZ = 1
nrows = g_nclust
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic, clust in enumerate(range(1, g_nclust+1)):
# for ic,clust in enumerate(row_clust_to_plot):
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
#         coords = df[(row_clusters==clust)].coords
        coords = df[(row_clusters==clust) & (df.z==z)].coords
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         poly_coords = df[(row_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        # hack iz hardcode
        img = vizutil.overlay_coords(back_img[iz], coords, list(cm_cycle(clust)[:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Row cluster {}, z={}".format(clust,z),fontsize=18)

In [ ]:
# nrows = len(col_clust_to_plot)
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic,clust in enumerate(range(1,g_nclust+1)):
# for ic,clust in enumerate(col_clust_to_plot):
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
        coords = df[(col_clusters==clust) & (df.z==z)].coords
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         coords = df[(col_clusters==clust)].coords
#         poly_coords = df[(col_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        img = vizutil.overlay_coords(back_img[iz], coords, list(cm_cycle(clust)[:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Col cluster {}, z={}".format(clust,z),fontsize=18)